In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
! pip install -q ragas
!pip -q install -U langchain-huggingface sentence-transformers

In [5]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("RAG").strip()

In [6]:
%run "/content/drive/Othercomputers/My Laptop/multimodal-rag/nootebooks/online_flow.ipynb"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f0d0b91971fe36047b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
import ast, json
import pandas as pd
from tqdm import tqdm

from ragas import EvaluationDataset, evaluate

from ragas.metrics.collections import (
    Faithfulness,
    AnswerRelevancy,
    ContextRecall,
    FactualCorrectness,
)

from ragas.metrics import NonLLMContextRecall

from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_huggingface import HuggingFaceEmbeddings

/tmp/ipython-input-2669908488.py:14: DeprecationWarning: Importing NonLLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import NonLLMContextRecall
  from ragas.metrics import NonLLMContextRecall


In [15]:
df = pd.read_csv("/content/drive/Othercomputers/My Laptop/multimodal-rag/testset/testset_text_law.csv")

def parse_list(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []
    if isinstance(x, list):
        return x
    s = str(x)
    try:
        return json.loads(s)
    except Exception:
        return ast.literal_eval(s)

In [16]:
# -------------------------
# 1) Adapter: chạy pipeline RAG của bạn để lấy (answer, contexts)
#    NOTE: Notebook online_flow.ipynb đã được %run ở cell trước, nên các biến/hàm như:
#    - rag_chain
#    - run_rag_multimodal_lcel(...)
#    ... sẽ có sẵn trong namespace hiện tại.
# -------------------------

def _extract_contexts_from_docs(docs):
    if not docs:
        return []
    ctxs = []
    for d in docs:
        # LangChain Document
        if hasattr(d, "page_content"):
            ctxs.append(d.page_content)
        # dict-like
        elif isinstance(d, dict):
            ctxs.append(d.get("page_content") or d.get("content") or str(d))
        else:
            ctxs.append(str(d))
    return ctxs

def run_rag(question: str):
    # Nếu pipeline của bạn trả ra dict như {"answer": ..., "docs": [...]}
    out = run_rag_multimodal_lcel(question, img_bytes=None, chain=rag_chain)

    answer = out.get("answer") or out.get("response") or out.get("output_text") or ""
    docs = (
        out.get("docs")
        or out.get("context")
        or out.get("source_documents")
        or out.get("retrieved_docs")
        or []
    )
    contexts = _extract_contexts_from_docs(docs)

    # fallback: nếu out đã có retrieved_contexts sẵn dạng list[str]
    if not contexts and isinstance(out.get("retrieved_contexts"), list):
        contexts = out["retrieved_contexts"]

    return answer, contexts


# -------------------------
# 2) Build EvaluationDataset
#    testset cần tối thiểu: user_input
#    nếu có: reference, reference_contexts thì dùng thêm cho metrics liên quan
# -------------------------
rows = []
for _, r in tqdm(df.iterrows(), total=len(df)):
    response, retrieved_contexts = run_rag(r["user_input"])
    rows.append(
        {
            "user_input": r["user_input"],
            "response": response,
            "retrieved_contexts": retrieved_contexts,
            "reference": r.get("reference", None),
            "reference_contexts": parse_list(r.get("reference_contexts", "[]")),
        }
    )

eval_ds = EvaluationDataset.from_list(rows)


# -------------------------
# 3) Evaluator LLM + Vietnamese embeddings (AITeamVN/Vietnamese_Embedding)
# -------------------------
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))

hf_embeddings = HuggingFaceEmbeddings(
    model_name="AITeamVN/Vietnamese_Embedding",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

evaluator_embeddings = LangchainEmbeddingsWrapper(hf_embeddings)


# -------------------------
# 4) Metrics + evaluate
# -------------------------
metrics = [
    Faithfulness(llm=evaluator_llm),
    AnswerRelevancy(llm=evaluator_llm, embeddings=evaluator_embeddings),
    ContextRecall(llm=evaluator_llm),
    FactualCorrectness(llm=evaluator_llm),
    NonLLMContextRecall(),  # dùng reference_contexts + retrieved_contexts
]

result = evaluate(
    eval_ds,
    metrics=metrics,
    llm=evaluator_llm,
    embeddings=evaluator_embeddings,
)

print(result)
scores = result.to_pandas()
scores.to_csv("ragas_scores.csv", index=False)
scores.head()


  0%|          | 0/5 [00:00<?, ?it/s]

🖼️ image: qid=None matched=None top2_sim=None

🔎 Multi-queries:
1. Xe ưu tiên là gì và có những loại nào trong quy định giao thông đường bộ?
2. Xe ưu tiên là gì và các loại xe được quy định giao thông
3. định nghĩa xe ưu tiên và những loại xe được quyền ưu tiên trong giao thông đường bộ

✅ Reranked docs: 10

attach_img = False


 20%|██        | 1/5 [00:26<01:45, 26.42s/it]


✅ ANSWER:
 Xe ưu tiên gồm xe chữa cháy của Cảnh sát phòng cháy, chữa cháy và cứu nạn, cứu hộ; xe của lực lượng quân sự, công an và kiểm sát đi làm nhiệm vụ khẩn cấp; đoàn xe có xe Cảnh sát giao thông dẫn đường; xe cứu thương đi làm nhiệm vụ cấp cứu; xe hộ đê đi làm nhiệm vụ; xe đi làm nhiệm vụ cứu nạn, cứu hộ, khắc phục sự cố thiên tai, dịch bệnh hoặc xe đi làm nhiệm vụ trong tình trạng khẩn cấp theo quy định của pháp luật; đoàn xe tang.
🖼️ image: qid=None matched=None top2_sim=None

🔎 Multi-queries:
1. Chủ đầu tư dự án xây dựng có trách nhiệm gì sau khi hoàn thành công trình đường bộ để bồi hoàn?
2. trách nhiệm của chủ đầu tư sau khi hoàn thành công trình đường bộ
3. nghĩa vụ bồi hoàn của chủ đầu tư sau khi xây dựng công trình đường bộ

✅ Reranked docs: 10

attach_img = False


 40%|████      | 2/5 [00:37<00:52, 17.35s/it]

...


✅ ANSWER:
 Có, xe ô tô của người nước ngoài đăng ký tại nước ngoài có tay lái ở bên phải tham gia giao thông tại Việt Nam phải chấp hành quy định của pháp luật về trật tự, an toàn giao thông đường bộ và phải được cơ quan có thẩm quyền cấp phép.
🖼️ image: qid=None matched=None top2_sim=None

🔎 Multi-queries:
1. chất ma túy có bị cấm khi lái xe không?
2. hành vi bị nghiêm cấm chất ma túy khi điều khiển phương tiện
3. lái xe có chất ma túy trong cơ thể có bị cấm theo quy định giao thông không

✅ Reranked docs: 10

attach_img = False


100%|██████████| 5/5 [01:04<00:00, 12.82s/it]


✅ ANSWER:
 Có, điều khiển phương tiện tham gia giao thông đường bộ mà trong cơ thể có chất ma túy hoặc chất kích thích khác mà pháp luật cấm sử dụng là hành vi bị nghiêm cấm.



/tmp/ipython-input-1638381148.py:69: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))


NameError: name 'torch' is not defined